In [1]:
!pip install geopy
from sklearn import preprocessing
import numpy as np
import pandas as pd
import glob
import geopy
import geopy.distance
import datetime
import sys
import time
#Training part of the data

from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer

import matplotlib.pyplot as plt

!pip install hyperopt
from hyperopt import fmin, tpe, hp,STATUS_OK, Trials

!pip install tpot
!pip install xgboost
#pip install IProgress
#!pip install ipywidgets
!pip install --upgrade jupyter_client
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#Tuning using genetic algorithim using TPOT library. install using ' pip install tpot ''

from tpot import TPOTRegressor
#from ipywidgets import IntProgres

  Using cached jupyter_client-6.1.7-py3-none-any.whl (108 kB)
  Attempting uninstall: jupyter-client
    Found existing installation: jupyter-client 6.1.5
    Uninstalling jupyter-client-6.1.5:
      Successfully uninstalled jupyter-client-6.1.5
Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


/opt/conda/lib/python3.8/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [37]:
# #only works with google hosted runtime 
# from google.colab import drive
# drive.flush_and_unmount()
# drive.mount('/content/drive')

In [2]:
#path = r"/content/drive/My Drive/ship_data_files /final_ship_data.csv"    # for hosted runtime use this path 

path = r"final_ship_data.csv" #for local runtime

                #remove lat2 ,lon2 
                
df=pd.read_csv(path,sep=',',low_memory=False)
test_data=df
df

,Length,Breadth,Draught,Longitude,Latitude,SOG,ETA,Distance remaining,speed
0,152.0,24.0,6.8,5.838777,53.63944,14.3,12.685507,440.630956,34.734991
1,152.0,24.0,6.8,5.845800,53.64076,14.3,12.667177,439.857752,34.724213
2,152.0,24.0,6.8,5.852149,53.64198,14.3,12.650496,439.158871,34.714754
3,152.0,24.0,6.8,5.858515,53.64320,14.3,12.633836,438.458162,34.705070
4,152.0,24.0,6.8,5.864876,53.64441,14.3,12.617179,437.758031,34.695397
...,...,...,...,...,...,...,...,...,...
186520,229.0,38.0,13.3,9.806900,53.55355,7.4,0.083519,1.808408,21.652544
186521,229.0,38.0,13.3,9.810224,53.55293,7.3,0.066774,1.434516,21.483189
186522,229.0,38.0,13.3,9.813442,53.55230,7.2,0.049974,1.071975,21.450709
186523,229.0,38.0,13.3,9.816650,53.55172,7.1,0.033443,0.711492,21.275079


In [3]:
N_ESTIMETOR=30

In [4]:
train=test_data

#Features
X_Train = train.drop('ETA', axis=1)

# Create our imputer to replace missing values with the mean e.g.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp = imp.fit(X_Train)


# Impute our data, then train
X_train_imp = imp.transform(X_Train)
X= np.array(X_train_imp)

# Labels
Y= np.array(train['ETA'])

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X)
X=X_train_minmax
Y.size

186525

In [5]:
X

array([[8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.85479736e-01, 2.87810730e-01],
       [8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.83925305e-01, 2.87694706e-01],
       [8.45771144e-02, 1.50684932e-01, 2.90598291e-01, ...,
        5.89958159e-01, 8.82520293e-01, 2.87592876e-01],
       ...,
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.92887029e-01, 1.80171753e-03, 1.44800810e-01],
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.88702929e-01, 1.07701073e-03, 1.42910086e-01],
       [2.12271973e-01, 3.42465753e-01, 8.46153846e-01, ...,
        2.88702929e-01, 3.63142472e-04, 1.44082390e-01]])

In [6]:
#Split train and test
train_features, test_features, train_labels, test_labels = train_test_split(X, Y, test_size = 0.25, random_state = 5)
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (139893, 8)
Training Labels Shape: (139893,)
Testing Features Shape: (46632, 8)
Testing Labels Shape: (46632,)


In [7]:
#Training of the model, for this example using Random forest regressor

start_time = time.time()
rf = RandomForestRegressor(verbose=10,n_estimators=N_ESTIMETOR,n_jobs=-1)

start_time = time.time()

rf.fit(train_features, train_labels);
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

time_noHpo= time.time() - start_time
test_features

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 30
building tree 2 of 30
building tree 3 of 30
building tree 4 of 30
building tree 5 of 30
building tree 6 of 30
building tree 7 of 30
building tree 8 of 30


[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.1s


building tree 9 of 30
building tree 10 of 30
building tree 11 of 30
building tree 12 of 30
building tree 13 of 30
building tree 14 of 30
building tree 15 of 30
building tree 16 of 30


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.2s


building tree 17 of 30
building tree 18 of 30
building tree 19 of 30
building tree 20 of 30


[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.2s


building tree 21 of 30
building tree 22 of 30
building tree 23 of 30
building tree 24 of 30
building tree 25 of 30
building tree 26 of 30
building tree 27 of 30
building tree 28 of 30
building tree 29 of 30
building tree 30 of 30


[Parallel(n_jobs=-1)]: Done  27 out of  30 | elapsed:    7.2s remaining:    0.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    8.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  17 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done  27 out of  30 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    0.3s finished


array([[0.17578773, 0.23287671, 0.2991453 , ..., 0.49790795, 0.02440506,
        0.29220158],
       [0.20563847, 0.24657534, 0.57264957, ..., 0.45188285, 0.12725067,
        0.1717478 ],
       [0.41293532, 0.45205479, 0.76068376, ..., 0.23849372, 0.43586908,
        0.27044907],
       ...,
       [0.29187396, 0.36986301, 0.60683761, ..., 0.79497908, 0.63529254,
        0.00296428],
       [0.13598673, 0.26027397, 0.44444444, ..., 0.43096234, 0.08142411,
        0.1362038 ],
       [0.13598673, 0.26027397, 0.52991453, ..., 0.46861925, 0.59828468,
        0.04423233]])

In [8]:
test_labels

array([ 0.35043611,  2.64979611,  6.55105444, ..., 38.20745083,
        1.96966306, 24.59149167])

In [9]:
mean1 = np.sqrt(mean_squared_error(predictions, test_labels))

print("%s seconds it took to train " % (time_noHpo))
print('\nRoot mean squared of Model without training', mean1)

8.595992088317871 seconds it took to train 

Root mean squared of Model without training 0.039063900099350214


In [10]:

# Use the random grid to search for best hyperparameters

rfh = RandomForestRegressor()
rfh.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [11]:
#Now to set the values needed for the parameters to do the random search, make combinations for the different parameters
#important HPO max features leave as auto, sample size> Low no.  Node size> Low values
max_depth=[1,32]
max_features = ['auto']
min_samples_leaf = [2]
min_samples_split = [ 5]
n_estimators = [int(x) for x in np.linspace(start = 10, stop =N_ESTIMETOR , num = 10)]
bootstrap=[True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
random_grid

{'n_estimators': [10, 12, 14, 16, 18, 21, 23, 25, 27, 30],
 'max_features': ['auto'],
 'max_depth': [1, 32],
 'min_samples_split': [5],
 'min_samples_leaf': [2],
 'bootstrap': [True, False]}

In [12]:
#Training of models with the tuning parameters made
start_time = time.time()


rf_random = RandomizedSearchCV(estimator = rfh, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=10, random_state=5, n_jobs =-1)
rf_random.fit(train_features, train_labels)

time_HpoRandom= time.time() - start_time

Fitting 3 folds for each of 40 candidates, totalling 120 fits


/opt/conda/lib/python3.8/site-packages/sklearn/model_selection/_search.py:278: UserWarning: The total space of parameters 40 is smaller than n_iter=100. Running 40 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   34.5s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done  77 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done  90 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 105 tasks      | elapsed:  2.7min
[Paral

In [13]:
rf_random.best_params_

{'n_estimators': 27,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 32,
 'bootstrap': True}

In [14]:
#Testing on the test features, predictions for random search
predictions2=rf_random.predict(test_features)


In [15]:
predictions2

array([ 0.35259787,  2.65237538,  6.54552048, ..., 38.16007788,
        1.96420695, 24.5889886 ])

In [16]:

mean2 =np.sqrt(mean_squared_error(predictions2, test_labels))
print("%s seconds it took to train " % (time_HpoRandom))
print('\nRoot mean squared of Model with Random search', mean2)

276.96853518486023 seconds it took to train 

Root mean squared of Model with Random search 0.04611791369544011


In [17]:
#Tuning with Bayesian optimization

#Define search space
# min leaf> Low values , min split> low values  ,  n_estimators > Higher number of trees
start_time = time.time()

search_space= {'max_depth':hp.choice('max_depth',range(1,32)),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform ('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0.1, 1.0),
        'n_estimators' : hp.choice('n_estimators', (10,N_ESTIMETOR))
    }

def objective(search_space):
    bayes = RandomForestRegressor(
                                 max_depth=search_space['max_depth'],
                                 max_features = search_space['max_features'],
                                 min_samples_leaf = search_space['min_samples_leaf'],
                                 min_samples_split = search_space['min_samples_split'],
                                 n_estimators = search_space['n_estimators'], 
                               )
    accuracy = cross_val_score(bayes, train_features, train_labels, cv =100).mean()
    return {'loss': -accuracy, 'status': STATUS_OK }

trials = Trials()
best = fmin(fn= objective,
            space= search_space,
            algo= tpe.suggest,
            max_evals = 20,
            trials= trials)
best

# Training from bayes tuning
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}

trainedforest = RandomForestRegressor( 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = best['n_estimators']
                                      ).fit(train_features, train_labels)
time_HpoBaysian= time.time() - start_time
predictions3= trainedforest.predict(test_features)

100%|██████████| 20/20 [22:19<00:00, 67.00s/trial, best loss: -0.8198714089553444] 


In [18]:
predictions3

array([ 1.81845335,  1.81845335,  5.60873907, ..., 30.34269669,
        1.81845335, 30.34269669])

In [19]:
mean3 = np.sqrt(mean_squared_error(predictions3, test_labels))
print("%s seconds it took to train " % (time_HpoBaysian))
print('\nRoot mean squared of Model with Bayesian', mean3)

1340.298006772995 seconds it took to train 

Root mean squared of Model with Bayesian 4.603309360833014


In [ ]:
best

In [20]:
start_time = time.time()
               
# Current values for object TPOTRegressor. Constructor has initial 100 generations

tpot_regressor= TPOTRegressor(generations=30, population_size=30,
                         offspring_size=12, mutation_rate=0.9,
                         crossover_rate=0.1,
                         scoring='neg_mean_squared_error', cv=5,
                         subsample=1.0, n_jobs=-1,
                         max_time_mins=None, max_eval_time_mins=5,
                         random_state=None, config_dict=None,
                         template=None,
                         warm_start=False,
                         memory=None,
                         use_dask=False,
                         periodic_checkpoint_folder=None,
                         early_stop=None,
                         verbosity=3,
                         disable_update_check=False)



tpot_regressor.fit(train_features, train_labels)
time_HpoTpot= time.time() - start_time 

30 operators have been imported by TPOT.


Skipped pipeline #20 due to time out. Continuing to the next pipeline.

Generation 1 - Current Pareto front scores:
-1	-0.009343290661664796	XGBRegressor(input_matrix, XGBRegressor__learning_rate=0.1, XGBRegressor__max_depth=8, XGBRegressor__min_child_weight=7, XGBRegressor__n_estimators=100, XGBRegressor__nthread=1, XGBRegressor__objective=reg:squarederror, XGBRegressor__subsample=0.2)
Skipped pipeline #46 due to time out. Continuing to the next pipeline.andomForestRegressor__bootstrap=False, RandomForestRegressor__max_features=0.5, RandomForestRegressor__min_samples_leaf=13, RandomForestRegressor__min_samples_split=14, RandomForestRegressor__n_estimators=100)

Generation 2 - Current Pareto front scores:
-1	-0.009343290661664796	XGBRegressor(input_matrix, XGBRegressor__learning_rate=0.1, XGBRegressor__max_depth=8, XGBRegressor__min_child_weight=7, XGBRegressor__n_estimators=100, XGBRegressor__nthread=1, XGBRegressor__objective=reg:squarederror, XGBRegressor__subsample=0.2)
-2	-0.00529

TPOTRegressor(generations=30,
              log_file=<ipykernel.iostream.OutStream object at 0x7f612bd6e640>,
              offspring_size=12, population_size=30,
              scoring='neg_mean_squared_error', verbosity=3)

In [21]:
predictionsTpot = tpot_regressor.predict(test_features)

In [22]:
predictionsTpot

array([ 0.35226329,  2.64602929,  6.55920907, ..., 38.19819018,
        1.96122257, 24.61914917])

In [23]:
mean4 =np.sqrt(mean_squared_error(predictionsTpot,test_labels))
print("%s seconds it took to train " % (time_HpoTpot)) #14 hours 
print('Root mean squared of Model with Genetic algorithim', mean4)

0.0009069442749023438 seconds it took to train 
Root mean squared of Model with Genetic algorithim 0.021548130102314367


In [25]:
print('Root mean squared of Model training without HPO', mean1)
print('Root mean squared of Model with Random search', mean2)
print('Root mean squared of Model with Bayesian', mean3)
print('Root mean squared of Model with Genetic algorithim', mean4)

Root mean squared of Model training without HPO 0.039063900099350214
Root mean squared of Model with Random search 0.04611791369544011
Root mean squared of Model with Bayesian 4.603309360833014
Root mean squared of Model with Genetic algorithim 0.021548130102314367
